In [2]:
!pip install langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [4]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.0 MB/s eta 0:00:00


In [12]:
#Import Modules
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
import google.generativeai as genai

# configure api key and initialise model
from google.colab import userdata
import os


if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('api_key')

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

#Initialize Model
llm = ChatGoogleGenerativeAI(model="gemini-pro")

#Load the blog
loader = WebBaseLoader("https://www.prothomalony.com/")
docs = loader.load()

#Define the Summarize Chain
summary = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""

prompt = PromptTemplate.from_template(summary)

llm_chain = LLMChain(llm=llm, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

#Invoke Chain
response = stuff_chain.invoke(docs)
print("Summary:\n", response["output_text"])

Summary:
 UK Prime Minister Rishi Sunak called for snap elections, with the election set for May 22nd. This unexpected move comes amid ongoing challenges in the job market, where women continue to face disparities compared to men in terms of opportunities and advancement.


In [18]:
named_entity = """Provide named entities in a presentative tabular form of the following:
"{text}"
NER:"""

prompt = PromptTemplate.from_template(named_entity)

llm_chain = LLMChain(llm=llm, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

#Invoke Chain
response=stuff_chain.invoke(docs)
print("NER:\n", response["output_text"])

NER:
 | Entity | Type |
|---|---|
| ঋষি সুনাক | Person |
| যুক্তরাজ্য | Country |
| নরওয়ে | Country |
| স্পেন | Country |
| আয়ারল্যান্ড | Country |
| ফিলিস্তিন | Country |
| কুয়েত | Country |
| বাংলাদেশ | Country |
| নুরুল ইসলাম | Person |
| হাঙ্গার ফ্রি এনওয়াইসি | Organization |
| এলিস মিউনরো | Person |
| হায়দার আকবর খান রনো | Person |
| নুরুল আলম লাল | Person |
| প্রগিতশীল গণ সংস্কৃতি পরিষদ (পিডিআই) | Organization |
| উদীচী শিল্পী গোষ্ঠী | Organization |
| আবদুল গাফ্ফার চৌধুরী | Person |
| আনার খুন | Person |
| ফারাক্কা বাঁধ | Infrastructure |
| দেয়ালের দেশ | Movie |
| ইব্রাহীম চৌধুরী | Person |


In [80]:
import csv
from collections import Counter

# Assuming the response is stored in the variable `response`
response_text = """
 | Entity | Type |
|---|---|
| ঋষি সুনাক | Person |
| যুক্তরাজ্য | Country |
| নরওয়ে | Country |
| স্পেন | Country |
| আয়ারল্যান্ড | Country |
| ফিলিস্তিন | Country |
| কুয়েত | Country |
| বাংলাদেশ | Country |
| নুরুল ইসলাম | Person |
| হাঙ্গার ফ্রি এনওয়াইসি | Organization |
| এলিস মিউনরো | Person |
| হায়দার আকবর খান রনো | Person |
| নুরুল আলম লাল | Person |
| প্রগিতশীল গণ সংস্কৃতি পরিষদ (পিডিআই) | Organization |
| উদীচী শিল্পী গোষ্ঠী | Organization |
| আবদুল গাফ্ফার চৌধুরী | Person |
| আনার খুন | Person |
| ফারাক্কা বাঁধ | Infrastructure |
| দেয়ালের দেশ | Movie |
| ইব্রাহীম চৌধুরী | Person |
"""

# Parse the response text to get entities and their types
lines = response_text.strip().split('\n')
data = []
types = []

for line in lines[3:]:  # Skip the header and separator lines
    parts = line.strip('|').split('|')
    if len(parts) == 2:
        entity = parts[0].strip()
        entity_type = parts[1].strip()
        data.append([entity, entity_type])
        types.append(entity_type)

# Count occurrences of each type
type_counts = Counter(types)

# Add counts to the data
data_with_counts = [[entity, entity_type, type_counts[entity_type]] for entity, entity_type in data]

# Write the data to a CSV file
csv_file = '/content/drive/MyDrive/Colab_Notebooks/llm_ner.csv'

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Entity', 'Type', 'Count'])
    writer.writerows(data_with_counts)

print(f"Data with counts has been written to {csv_file}")

Data with counts has been written to /content/drive/MyDrive/Colab_Notebooks/llm_ner.csv


In [42]:
topic = """Provide topic in a presentative tabular form of the following:
"{text}"
Topic:"""

prompt = PromptTemplate.from_template(topic)

llm_chain = LLMChain(llm=llm, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

#Invoke Chain
response=stuff_chain.invoke(docs)
print("Topic:\n", response["output_text"])

Topic:
 **Prothom Alo New York - প্রথম আলো - নিউইয়র্ক**

**উত্তর আমেরিকা**

| **শিরোনাম** | **তারিখ** | **প্রকার** | **লেখক** |
|---|---|---|---|
| হুট করেই যুক্তরাজ্যে আগাম নির্বাচনের ঘোষণা করলেন ঋষি সুনাক | ২৪ মে ২০২৪ | সংবাদ | N/A |
| চাকুরির বাজারে পুরুষের চেয়ে নারীরা পিছিয়ে কেন ? | N/A | মতামত | শারমিন আকতার |
| কা র্টে ল আর স্নে ই ক হে ডে র হাতে প্রাণ ঝ র ছে ইমিগ্র্যান্ট প্রত্যাশীদের | N/A | সংবাদ | N/A |
| বাংলায় ফুড স্ট্যাম্প সেবা চালু করল হাঙ্গার ফ্রি এনওয়াইসি | N/A | সংবাদ | N/A |
| কুয়েতে মুক্তিযোদ্ধা নুরুল ইসলামকে সংবর্ধনা | N/A | সংবাদ | N/A |
| জ্যামাইকা ফেন্ডস সোসাইটি: নিয়মনীতির তোয়াক্কা নেই | N/A | সংবাদ | N/A |
| নারায়ণগঞ্জ জেলা সমিতির ঐক্যবদ্ধ কমিটি গঠন সভাপতি তুহিন, সা. সম্পাদক টিটো | N/A | সংবাদ | N/A |
| ফি লি স্তি ন কে রাষ্ট্রের স্বীকৃতি দিচ্ছে নরওয়ে, স্পেন ও আয়ারল্যান্ড | N/A | সংবাদ | N/A |
| বৈশাখী রিভার ক্রুজে আমন্ত্রণ | N/A | সংবাদ | N/A |
| ঢাকা বিশ্ববিদ্যালয় অ্যালামনাই ইন দ্য ইউকে'র সাথে ঢাবি ভিসি প্রফেসর ডঃ এএসএম মাকসুদ কামালের মতবিনিময় | N/A | সংবাদ |

In [43]:
sentiment = """Provide sentiment in a presentative tabular form of the following:
"{text}"
Sentiment:"""

prompt = PromptTemplate.from_template(sentiment)

llm_chain = LLMChain(llm=llm, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

#Invoke Chain
response=stuff_chain.invoke(docs)
print("Sentiment:\n", response["output_text"])

Sentiment:
 | News Title | Sentiment |
|---|---|
| হুট করেই যুক্তরাজ্যে আগাম নির্বাচনের ঘোষণা করলেন ঋষি সুনাক | Neutral |
| চাকুরির বাজারে পুরুষের চেয়ে নারীরা পিছিয়ে কেন ? | Negative |
| কা র্টে ল আর স্নে ই ক হে ডে র হাতে প্রাণ ঝ র ছে ইমিগ্র্যান্ট প্রত্যাশীদের | Negative |
| বাংলায় ফুড স্ট্যাম্প সেবা চালু করল হাঙ্গার ফ্রি এনওয়াইসি | Positive |
| কুয়েতে মুক্তিযোদ্ধা নুরুল ইসলামকে সংবর্ধনা | Positive |
| জ্যামাইকা ফেন্ডস সোসাইটি: নিয়মনীতির তোয়াক্কা নেই | Negative |
| নারায়ণগঞ্জ জেলা সমিতির ঐক্যবদ্ধ কমিটি গঠন সভাপতি তুহিন, সা. সম্পাদক টিটো | Neutral |
| ফি লি স্তি ন কে রাষ্ট্রের স্বীকৃতি দিচ্ছে নরওয়ে, স্পেন ও আয়ারল্যান্ড | Positive |
| বৈশাখী রিভার ক্রুজে আমন্ত্রণ | Positive |
| ঢাকা বিশ্ববিদ্যালয় অ্যালামনাই ইন দ্য ইউকে'র সাথে ঢাবি ভিসি প্রফেসর ডঃ এএসএম মাকসুদ কামালের মতবিনিময় | Neutral |
| কিসের লজ্জা | Neutral |
| গ্রে প্তা রি প রো য়া না র আবেদন সত্ত্বেও যুক্তরাষ্ট্রে আমন্ত্রণ পাচ্ছেন নেতানিয়াহু | Neutral |


In [77]:
word_cloud = """Provide word cloud and counts of the following:
"{text}"
Word cloud:"""

prompt = PromptTemplate.from_template(word_cloud)

llm_chain = LLMChain(llm=llm, prompt=prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

#Invoke Chain
response=stuff_chain.invoke(docs)
print("Word cloud:\n", response["output_text"])

Word cloud:
 [New York, USA, Bangladesh, Canada, Literature, Sports, Entertainment, Fashion, Opinion, Video, Foundation, E-Paper, Latest, Popular, Op-ed, North America, Jamaica Funds Society, Narayanganj District Association, Palestine, Norway, Spain, Ireland, Vaiskhaki River Cruise, Dhaka University Alumni in the UK, Bangladesh Day Parade, Housing Authority, Queensboro Dance Festival, Amir Uzzaman Tutul, Saree goes global, Nobel Prize, Ellis Munro, Haider Akbar Khan Rono, Nurul Alam Lal, Paraben, Shilpa Ghar, Sharmeen Akhtar, Salimullah Khan, Sitangshu Guha, Romeen Laiys, Abdul Gaffar Chowdhury, Rafiqul Islam, Tofail Ahmed, Arafat Rahman Coco, Salahuddin Ahmed, Jahangir Alam Akash, Bangladesh Football Team, Farakka Barrage, Dejaller Desh, Ibrahim Chowdhury]

Word counts:
- New York: 5
- USA: 1
- Bangladesh: 3
- Canada: 2
- Literature: 2
- Sports: 2
- Entertainment: 2
- Fashion: 1
- Opinion: 3
- Video: 1
- Foundation: 1
- E-Paper: 1
- Latest: 1
- Popular: 1
- Op-ed: 1
- North America: 